In [1]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float 
import psycopg2

In [2]:
rds_connection_string = "<insert user name>:<insert password>@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
athletes_events_file = "../ETL/athlete_events.csv"
regions_file = "../ETL/noc_regions.csv"
ses_file = "../ETL/country_SES_1880-2010_1.csv"



In [4]:
athletes_df = pd.read_csv(athletes_events_file)
athletes_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [5]:
# Get a list of all of our columns for easy reference
athletes_df.columns

Index(['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games',
       'Year', 'Season', 'City', 'Sport', 'Event', 'Medal'],
      dtype='object')

In [6]:
regions_df = pd.read_csv(regions_file)
noc_df = pd.DataFrame(regions_df, columns = ["NOC", "region"])
noc_df.head()

,NOC,region
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra


In [7]:
olympic = pd.DataFrame(athletes_df, columns = ["Team", "NOC", "Year", "Season", "City", "Sport", "Event", "Medal"])
olympic.head()


,Team,NOC,Year,Season,City,Sport,Event,Medal
0,China,CHN,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,China,CHN,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,Denmark,DEN,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,Netherlands,NED,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [8]:
#Dropping non medal winners
#Renaming columns in lower case

olympic = olympic.dropna()
olympic = olympic.rename(columns={"Team":"team", "NOC":"country_id", "Year":"year", 
                                  "Season":"season", "City":"city", "Sport":"sport", 
                                  "Event":"event", "Medal":"medal"})
olympic.head()

,team,country_id,year,season,city,sport,event,medal
3,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
37,Finland,FIN,1920,Summer,Antwerpen,Swimming,Swimming Men's 200 metres Breaststroke,Bronze
38,Finland,FIN,1920,Summer,Antwerpen,Swimming,Swimming Men's 400 metres Breaststroke,Bronze
40,Finland,FIN,2014,Winter,Sochi,Ice Hockey,Ice Hockey Men's Ice Hockey,Bronze
41,Finland,FIN,1948,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,Bronze


In [9]:
#Droping columns which are not needed
#Renaming columns in lower case
#Renaming NOC and wbid columns to country_id to match across dataframes for merging purposes
ses_df = pd.read_csv(ses_file, encoding=  "ISO-8859-1")
ses = pd.DataFrame(ses_df, columns = ["wbid", "country", "year", "SES", "gdppc"])
ses = ses.rename(columns={"wbid": "country_id", "SES":"ses"})
ses.head()

,country_id,country,year,ses,gdppc
0,AFG,Afghanistan,1970,3.474212,709.00000
1,AFG,Afghanistan,1920,26.968016,731.75677
2,AFG,Afghanistan,1990,1.269530,604.00000
3,AFG,Afghanistan,1960,15.763076,739.00000
4,AFG,Afghanistan,2000,2.061114,565.00000


In [10]:
noc_df = noc_df.rename(columns={"NOC": "country_id"})
noc_df = noc_df.rename(columns={"region": "country"})
noc_df.head()

,country_id,country
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra


In [11]:
merged_olympic_df = pd.merge(olympic, noc_df, on="country_id")

merged_olympic_df.head()

,team,country_id,year,season,city,sport,event,medal,country
0,Denmark/Sweden,DEN,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark
1,Denmark,DEN,1920,Summer,Antwerpen,Gymnastics,"Gymnastics Men's Team All-Around, Free System",Gold,Denmark
2,Nykjbings paa Falster,DEN,1912,Summer,Stockholm,Rowing,"Rowing Men's Coxed Fours, Inriggers",Gold,Denmark
3,Denmark,DEN,1906,Summer,Athina,Football,Football Men's Football,Gold,Denmark
4,Denmark,DEN,1908,Summer,London,Wrestling,"Wrestling Men's Middleweight, Greco-Roman",Bronze,Denmark
